In [ ]:
import os
os.listdir()
from google.colab import drive
drive.mount('/content/drive/')
os.listdir("./drive/MyDrive/projs/rep_sp")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


['data', 'polnear_finetuner.ipynb']

In [ ]:
#@title Package imports etc
#https://docs.databricks.com/en/_extras/notebooks/source/deep-learning/tune-classification-model-hugging-face-transformers.html
#https://github.com/zo-fi/slo_wsd_ZFMA
#https://stackoverflow.com/questions/76448287/how-can-i-solve-importerror-using-the-trainer-with-pytorch-requires-accele
! pip install -U accelerate
! pip install -U transformers

!pip install evaluate
!pip install datasets

import pandas as pd
import xml.etree.ElementTree as ET
import re
import os

from google.colab import drive
drive.mount('/content/drive/')
import numpy as np
from tqdm import tqdm
import seaborn as sns
import nltk
import json
#import classla
from tqdm import tqdm
from tqdm.gui import tqdm as tqdm_gui
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score

from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments, EarlyStoppingCallback, IntervalStrategy
import evaluate
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

import torch
cuda_test = torch.cuda.is_available()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
import random
random.seed(101)


Mounted at /content/drive/


In [ ]:
rwg = pd.read_csv("drive/MyDrive/projs/rep_sp/data/rwg_processed.csv")
riq = pd.read_csv("drive/MyDrive/projs/rep_sp/data/riqua_processed.csv")

rwg_train = rwg[rwg["split"] == "train"]
rwg_dev = rwg[rwg["split"] == "dev"]
riq_train = riq[riq["split"] == "train"]
riq_dev = riq[riq["split"] == "dev"]

train_df = pd.concat([rwg_train, riq_train])
dev_df = pd.concat([riq_train, riq_dev])

"sents", "quote_tag"
train_df.rename(columns = {"sents":"example", "quote_tag":"label"}, inplace = True)
train_df["label"] = train_df.label.apply(int)

dev_df.rename(columns = {"sents":"example", "quote_tag":"label"}, inplace = True)
dev_df["label"] = dev_df.label.apply(int)


train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(dev_df)

datasets = DatasetDict({"train" : train_dataset, "validation" : val_dataset})#, "test": test_dataset

#data = "drive/MyDrive/projs/rep_sp/data/polnear_processed.csv"
#df = pd.read_csv(data)
#df.rename(columns = {"sent":"example", "quote_tag":"label"}, inplace = True)

#train_df = df[df.role == "train"]#
#val_df = df[df.role == "dev"]
#test_df = df[df.role == "test"]


#loading the data properly

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

datasets = DatasetDict({"train" : train_dataset, "validation" : val_dataset, "test": test_dataset})




In [ ]:
#@title Load model, tokenizer
#"FacebookAI/xlm-roberta-base"
tgt_model = "FacebookAI/xlm-roberta-base"#"markussagen/xlm-roberta-longformer-base-4096"#"FacebookAI/xlm-roberta-base" #"google-bert/bert-base-multilingual-cased"
model = AutoModelForSequenceClassification.from_pretrained(tgt_model, return_dict=True, num_labels=2)
freeze_list = ["bert.encoder.layer.{}".format(str(i)) for i in range(0, 8)]
for name, param in model.named_parameters():
    if any([name.startswith(ban_param) for ban_param in freeze_list]):
        param.requires_grad = False

tokenizer = AutoTokenizer.from_pretrained(tgt_model)
def encode(examples):
  out = tokenizer(examples["example"], truncation = True, padding = "max_length") #, max_length = MAX_LEN
  return out

encoded_dataset = datasets.map(encode, batched=True)
#encoded_dataset.set_format(columns=['attention_mask', 'input_ids', 'token_type_ids', 'label'])
#label2id=label2id, id2label=id2label?


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/46064 [00:00<?, ? examples/s]

Map:   0%|          | 0/31586 [00:00<?, ? examples/s]

In [ ]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['example', 'sent_id', 'label', 'src', 'role', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 28297
    })
    validation: Dataset({
        features: ['example', 'sent_id', 'label', 'src', 'role', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3077
    })
    test: Dataset({
        features: ['example', 'sent_id', 'label', 'src', 'role', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2779
    })
})

In [ ]:
#!pip install accelerate -U

metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def compute_metrics(predic_obj):
  logits = predic_obj.predictions
  labels = predic_obj.label_ids
  predictions = np.argmax(logits, axis=-1)
  return metrics.compute(predictions=predictions, references=labels)

training_output_dir = "drive/MyDrive/projs/rep_sp/train_dump/"
training_args = TrainingArguments(
    output_dir = training_output_dir,
    evaluation_strategy="epoch",
    num_train_epochs = 2,
    gradient_accumulation_steps = 16,
    auto_find_batch_size = True)

data_collator = DataCollatorWithPadding(tokenizer)

trainer = Trainer(
    args=training_args,
    data_collator=DataCollatorWithPadding(tokenizer),
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    model=model#, callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)


trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,No log,0.248116
1,0.335000,0.179897


TrainOutput(global_step=718, training_loss=0.3065944129710078, metrics={'train_runtime': 9870.5215, 'train_samples_per_second': 9.334, 'train_steps_per_second': 0.073, 'total_flos': 2.418095843180544e+16, 'train_loss': 0.3065944129710078, 'epoch': 1.9951372004168113})

In [ ]:
model.eval()
preds = trainer.predict(encoded_dataset["test"])
compute_metrics(preds)

#compute_metrics

#metrics={'test_loss': 0.3038247525691986,
# 	Training Loss 	Validation Loss
#0 	No log 	0.320669
#1 	No log 	0.317925
#TrainOutput(global_step=328, training_loss=0.35928954147711034, metrics={'train_runtime': 3263.6252, 'train_samples_per_second': 12.886, 'train_steps_per_second': 0.101, 'total_flos': 1.10454021040128e+16, 'train_loss': 0.35928954147711034, 'epoch': 1.9961962723469})

{'accuracy': 0.7117668225980569,
 'f1': 0.7971638389465687,
 'precision': 0.709009009009009,
 'recall': 0.9103528050896472}

In [ ]:
tgt_path = "./drive/MyDrive/projs/rep_sp/models/no_polnear_xlmr"
os.mkdir(tgt_path)
model.save_pretrained(tgt_path)

In [ ]:

#os.listdir("./drive/MyDrive/projs/rep_sp/models")

In [ ]:
#Data exploration
#a lot of fictional data actually, maybe separate testig would be a good idea
#I don't really wanna throw away the data
df.columns
df.fictional.value_counts()
df[df.fictional == "no"].quote_tag.value_counts()
df[df.fictional == "yes"].quote_tag.value_counts()
df_train.quote_tag.value_counts()
df_test.quote_tag.value_counts()